# Mount Drive 




In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

Config option `delete_to_trash` not recognized by `ColabFileContentsManager`.
Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret
google.colab serverextension initialized.
Serving notebooks from local directory: /
0 active kernels
The Jupyter Notebook is running at:
http://172.28.0.2:9000/
Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
Kernel started: 83bd10c9-dcf0-406e-959f-ce766481724e
Adapting to protocol v5.1 for kernel 83bd10c9-dcf0-406e-959f-ce766481724e




#Load Required Packages

In [6]:
!pip install datasets
!pip install transformers
!pip3 install unidecode
!pip install pyserini==0.10.0.0

     |████████████████████████████████| 63.1MB 75kB/s 
     |████████████████████████████████| 1.1MB 50.1MB/s 


In [7]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 26.3 GB  |     Proc size: 117.4 MB
GPU RAM Free: 16130MB | Used: 0MB | Util   0% | Total     16130MB


In [8]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import join
import unidecode
import re
import logging
from tqdm import tqdm_notebook
from tqdm.notebook import tnrange
import glob
import json
import multiprocessing
from pyserini.search import SimpleSearcher
import time

#For ploting results
import matplotlib.pyplot as plt

# DL Libraries
from transformers import BertModel, AdamW, BertTokenizer, BertConfig, RobertaTokenizer, RobertaModel
from keras.preprocessing.sequence import pad_sequences
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, f1_score

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

# PYSERINI SETUP Files
(No need to execute)

**(Only need to be executed once.)**

Usings existing scipts provided by anserini for MSMARCO dataset.

In [11]:
#!git clone https://github.com/castorini/anserini-tools.git

In [12]:
#%cd /content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/
#!ls

In [13]:
#%cd /content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage 

In [14]:
#!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz

In [15]:
#!tar xvfz collectionandqueries.tar.gz

In [16]:
#%cd /content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/
#!ls

To generate 'lucene-index' for MSMARCO.

In [17]:
#!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
# -threads 9 -input collections/msmarco-passage/collection_jsonl \
# -index indexes/msmarco-passage/lucene-index-msmarco -storePositions -storeDocvectors -storeRaw

Reformating appropriately.

In [18]:
#!python scripts/msmarco/convert_collection_to_jsonl.py \
# --collection-path collections/msmarco-passage/collection.tsv \
# --output-folder collections/msmarco-passage/collection_jsonl

In [19]:
#!python scripts/msmarco/filter_queries.py \
# --qrels collections/msmarco-passage/qrels.dev.small.tsv \
# --queries collections/msmarco-passage/queries.dev.tsv \
# --output collections/msmarco-passage/queries.dev.small.tsv

In [20]:
#!python scripts/msmarco/retrieve.py --hits 50 --threads 1 \
# --index indexes/msmarco-passage/lucene-index-msmarco \
# --queries collections/msmarco-passage/queries.dev.small.tsv \
# --output runs/run.msmarco-passage.dev.small.tsv

Standard MSMARCO evaluation (MRR @10)

# Baseline Evaluation

In [21]:
%cd /content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/
!ls

/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools
collections  eval  indexes  README.md  runs  scripts  topics-and-qrels


In [22]:
!python scripts/msmarco/msmarco_eval.py \
 collections/msmarco-passage/qrels.dev.small.tsv runs/run.msmarco-passage.dev.small.tsv

#####################
MRR @10: 0.18741227770955546
QueriesRanked: 6980
#####################


TREC Evaluation metrics:

In [23]:
!python scripts/msmarco/convert_msmarco_to_trec_run.py \
 --input runs/run.msmarco-passage.dev.small.tsv \
 --output runs/run.msmarco-passage.dev.small.trec

!python scripts/msmarco/convert_msmarco_to_trec_qrels.py \
 --input collections/msmarco-passage/qrels.dev.small.tsv \
 --output collections/msmarco-passage/qrels.dev.small.trec

Done!
Done!


In [36]:
%cd /content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/eval/trec_eval.9.0.4/ 
!chmod +x *
!make
!make quicktest

/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/eval/trec_eval.9.0.4
make: 'trec_eval' is up to date.
./trec_eval test/qrels.test test/results.test | diff - test/out.test
./trec_eval -m all_trec test/qrels.test test/results.test | diff - test/out.test.a
./trec_eval -m all_trec -q test/qrels.test test/results.test | diff - test/out.test.aq
./trec_eval -m all_trec -q -c test/qrels.test test/results.trunc | diff - test/out.test.aqc
./trec_eval -m all_trec -q -c -M100 test/qrels.test test/results.trunc | diff - test/out.test.aqcM
./trec_eval -m all_trec -mrelstring.20 -q -l2 test/qrels.rel_level test/results.test | diff - test/out.test.aql
./trec_eval -m all_prefs -q -R prefs test/prefs.test test/prefs.results.test | diff - test/out.test.prefs
./trec_eval -m all_prefs -q -R qrels_prefs test/qrels.test test/results.test | diff - test/out.test.qrels_prefs
./trec_eval -m qrels_jg -q -R qrels_jg  test/qrels.123 test/results.test | diff - test/out.test.qrels_jg
./trec_eval -q -mip

In [47]:
!./trec_eval "-c" "-m" "P.5" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage.dev.small.trec"
!./trec_eval "-c" "-m" "P.10" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage.dev.small.trec"
!./trec_eval "-c" "-m" "recall.5" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage.dev.small.trec"
!./trec_eval "-c" "-m" "recall.10" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage.dev.small.trec"
!./trec_eval "-c" "-m" "recall.20" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage.dev.small.trec"
!./trec_eval "-c" "-m" "map" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage.dev.small.trec"
!./trec_eval "-c" "-m" "recip_rank"  "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage.dev.small.trec"

P_5                   	all	0.0611
P_10                  	all	0.0408
recall_5              	all	0.2944
recall_10             	all	0.3916
recall_20             	all	0.4842
map                   	all	0.1937
recip_rank            	all	0.1974


# SIA Evaluation


In [ ]:
total_start_time = time.time()
searcher.set_bm25(0.82, 0.68)

with open('/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage_SIA.dev.small.tsv', 'w') as fout:
    start_time = time.time()
    for line_number, line in enumerate(open('/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/queries.dev.small.tsv', 'r', encoding='utf8')):
        qid, query = line.strip().split('\t')
        hits = searcher.search(query, 50)
        if line_number % 100 == 0:
            time_per_query = (time.time() - start_time) / (line_number + 1)
            print(f'Retrieving query {line_number} ({time_per_query:0.3f} s/query)', flush=True)
        for rank in range(len(hits)):
            docno = hits[rank].docid
            doc = searcher.doc(str(hits[rank].docid))
            cleanDoc = re.findall(r'"(.*?)"', doc.raw())[3]
            fout.write('{}\t{}\t{}\t{}\t{}\n'.format(qid, docno, query, cleanDoc, rank + 1))
total_time = (time.time() - total_start_time)
print(f'Total retrieval time: {total_time:0.3f} s')
print('Done!')

In [ ]:
num_processes = multiprocessing.cpu_count()

def chunker(df):
  chunk_size = int(df.shape[0]/num_processes)
  return [df.loc[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]

def vectorize(d):
    tokenized = d.apply((lambda x: tokenizer.encode_plus(x.Query, x.Passage, max_length=512, add_special_tokens=True, return_token_type_ids=False, 
                                       padding='max_length', return_attention_mask=True, truncation=True)), axis=1)
    return tokenized

def dataloader(d):
    ip_id = []
    masks = []

    for da in d:
      for i in da:
        ip_id.append(i['input_ids'])
        masks.append(i['attention_mask'])

    inputs = torch.tensor(ip_id).to(device)
    mask = torch.tensor(masks).to(device)
    data = TensorDataset(inputs, mask)
    dataloader = DataLoader(data, batch_size=4)

    return dataloader

def sts_score_generator(ten, model):
  scores = []
  for batch in tqdm_notebook(ten):
    batch = tuple(t.to(device) for t in batch)
    ip_ids,mask = batch
    score = model(ip_ids, attention_mask = mask)
    score = score.squeeze(1)
    scores.extend(score.detach().cpu().numpy())
  return scores

def PredictCustom(df):
  pool = multiprocessing.Pool(processes=num_processes)
  print("*")
  chunks = chunker(df)
  print("**")
  res = pool.map(vectorize, chunks)
  print("***")
  result = dataloader(res)
  print("****")
  return sts_score_generator(result, customModel)

class Regressor(nn.Module):

  def __init__(self):
    super(Regressor, self).__init__()
    self.bert = RobertaModel.from_pretrained('roberta-base')
    self.out = nn.Linear(self.bert.config.hidden_size, 1)

  def forward(self, input_ids, attention_mask):
    output, pooler_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    score= self.out(pooler_out)
    return score

def SaveFixedLabel(df, name):
  df.to_csv('/content/drive/My Drive/Sahit/work/project_nlp/data/' + name + '.tsv')


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
customModel = torch.load('/content/drive/My Drive/project_nlp/sia_experiment/modelMSMARCO/pytorch_modelTrained').to(device)

In [ ]:
dfEval = pd.read_csv('/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage_SIA.dev.small.tsv', delimiter='\t', header=None)
dfEval.columns = ['qid', 'docno', 'Query', 'Passage', 'Ranks']
dfEval

In [ ]:
dfEval['sia_score'] = PredictCustom(dfEval)

Re-ranking Top 20 passages from Top 50 by BM25 usin gour SIA model.

In [ ]:
dfEval.sort_values(['sia_score'],ascending=False).groupby('qid').head(20)
dfEvalRank = dfEval.groupby('qid').rank(ascending=False)
dfEvalRank.columns = ['r1', 'r2', 'rank']
res = pd.concat([dfEval, dfEvalRank], axis=1)
res[['rank']] = res[['rank']].astype(int)
res = res.sort_values(['sia_score'],ascending=False).groupby('qid').head(20)
res = res.sort_values(['qid', 'sia_score'],ascending=False)
res

In [ ]:
res[['qid', 'docno', 'rank']].to_csv('/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage_SIA_Top50.dev.small.tsv',  sep="\t", header=False, index=False)

SIA model evaluaiton on: MRR @10

In [28]:
%cd /content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/
!ls

/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools
collections  eval  indexes  README.md  runs  scripts  topics-and-qrels


In [30]:
!python scripts/msmarco/msmarco_eval.py \
 collections/msmarco-passage/qrels.dev.small.tsv runs/run.msmarco-passage_SIA_Top50.dev.small.tsv

#####################
MRR @10: 0.200630145085732
QueriesRanked: 6980
#####################


SIA model evaluation on TREC metrics:

In [ ]:
!python scripts/msmarco/convert_msmarco_to_trec_run.py \
 --input runs/run.msmarco-passage_SIA_Top50.dev.small.tsv \
 --output runs/run.msmarco-passage_SIA_Top50.dev.small.trec

!python scripts/msmarco/convert_msmarco_to_trec_qrels.py \
 --input collections/msmarco-passage/qrels.dev.small.tsv \
 --output collections/msmarco-passage/qrels.dev.small.trec

In [37]:
%cd /content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/eval/trec_eval.9.0.4/ 

/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/eval/trec_eval.9.0.4


In [33]:
!ls

collections  eval  indexes  README.md  runs  scripts  topics-and-qrels


In [46]:
!./trec_eval "-c" "-m" "P.5" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage_SIA_Top50.dev.small.trec"
!./trec_eval "-c" "-m" "P.10" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage_SIA_Top50.dev.small.trec"
!./trec_eval "-c" "-m" "recall.5" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage_SIA_Top50.dev.small.trec"
!./trec_eval "-c" "-m" "recall.10" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage_SIA_Top50.dev.small.trec"
!./trec_eval "-c" "-m" "recall.20" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage_SIA_Top50.dev.small.trec"
!./trec_eval "-c" "-m" "map" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage_SIA_Top50.dev.small.trec"
!./trec_eval "-c" "-m" "recip_rank"  "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/collections/msmarco-passage/qrels.dev.small.trec" "/content/drive/My Drive/Sahit/work/project_nlp/anserini-tools/runs/run.msmarco-passage_SIA_Top50.dev.small.trec"

P_5                   	all	0.0645
P_10                  	all	0.0423
recall_5              	all	0.3103
recall_10             	all	0.4058
recall_20             	all	0.4981
map                   	all	0.2027
recip_rank            	all	0.2070


# Results

Baseline BM25:
```
P @5                   	0.0611
P @10                  	0.0408
Recall @5              	0.2944
Recall @10             	0.3916
Recall @20             	0.4842
MAP                   	0.1937
MRR                   	0.1974
```
SIA model:


```
P @5                    0.0645
P @10                  	0.0423
Recall @5              	0.3103
Recall @10             	0.4058
Recall @20             	0.4981
MAP                   	0.2027
MAR            	        0.2070
```




